# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Attention
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from transformer_build import  *
from data_preprocessing import *
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-train-stemmed.csv')
valid_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-dev-stemmed.csv')
test_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-test-stemmed.csv')
train_df.head()

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


In [3]:
# train_df와 valid_df 결합
train_val_df = pd.concat([train_df, valid_df])

In [4]:
textcuboid=np.load('./1-Channel textcuboid_r8(elmo).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_r8(elmo).npy')

# 전처리

In [5]:
train_val_df['text']=train_val_df['text'].apply(clean_text)
test_df['text']=test_df['text'].apply(clean_text)

In [6]:
x_train=list(train_val_df['text'])
y_train=list(train_val_df['intent'])
x_test=list(test_df['text'])
y_test=list(test_df['intent'])

In [7]:
y=y_train+y_test

In [8]:
encoder=LabelEncoder()
encoder.fit(y)
label=encoder.transform(y)

y_train=list(label[:5484])
y_test=list(label[5484:])

In [9]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test


tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [10]:
xtext_train.shape

(5484, 300)

In [11]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [12]:
textcuboid_train=textcuboid[:4937]
textcuboid_val=textcuboid[4937:]
text_train=xtext_train[:4937]
text_val=xtext_train[4937:]
y_train1=y_train[:4937]
y_val=y_train[4937:]

In [13]:
text_test=xtext_test
y_test1=y_test

# 모델 빌드


In [14]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, save_freq=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [15]:
embedding_dim = 256 # 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1

#ELMo channel의 학습
ELMo_input = Input(shape=(237, 256))
conv1 = Conv1D(256, 1, padding='valid', activation='relu')(ELMo_input)
pooling = GlobalMaxPooling1D()(conv1)
pooling = Dropout(0.5)(pooling)
ELMo_x = Dense(256, activation='relu')(pooling)
# ELMo_attention_layer = Attention()
# ELMo_a=ELMo_attention_layer([ELMo_x,ELMo_x])

#Sequence channel의 학습
Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    Text_x = transformer_block(Text_x)

Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)
# Text_attention_layer = Attention()
# Text_a=Text_attention_layer([Text_x,Text_x])

#결합 후 분류
# x = layers.concatenate([ELMo_x,ELMo_a, Text_x,Text_a], axis=-1)
# x = layers.concatenate([ELMo_a, Text_a], axis=-1)
x = layers.concatenate([ELMo_x, Text_x], axis=-1)
# x = Dropout(0.5)(x)
output_layer = Dense(8, activation='softmax')(x)

model = Model(inputs=[ELMo_input, Text_inputs], outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 237, 256)]   0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 300, 256)    4528384     ['input_2[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 conv1d (Conv1D)                (None, 237, 256)     65792       ['input_1[0][0]']            

# 모델 학습

In [16]:
#모델 학습
history = model.fit([textcuboid_train,text_train], y_train1,callbacks=callbacks, epochs=50,batch_size=256,validation_data=([textcuboid_val,text_val], y_val))

Epoch 1/50
20/20 [==============================] - 3s 67ms/step - loss: 1.2184 - accuracy: 0.6504 - val_loss: 0.6582 - val_accuracy: 0.8154 - lr: 0.0010
Epoch 2/50
20/20 [==============================] - 1s 56ms/step - loss: 0.5055 - accuracy: 0.8444 - val_loss: 0.3894 - val_accuracy: 0.8848 - lr: 0.0010
Epoch 3/50
20/20 [==============================] - 1s 55ms/step - loss: 0.3133 - accuracy: 0.8987 - val_loss: 0.2877 - val_accuracy: 0.9232 - lr: 0.0010
Epoch 4/50
20/20 [==============================] - 1s 56ms/step - loss: 0.2033 - accuracy: 0.9346 - val_loss: 0.1998 - val_accuracy: 0.9342 - lr: 0.0010
Epoch 5/50
20/20 [==============================] - 1s 56ms/step - loss: 0.0995 - accuracy: 0.9680 - val_loss: 0.1343 - val_accuracy: 0.9506 - lr: 0.0010
Epoch 6/50
20/20 [==============================] - 1s 54ms/step - loss: 0.0357 - accuracy: 0.9876 - val_loss: 0.1521 - val_accuracy: 0.9561 - lr: 0.0010
Epoch 7/50
20/20 [==============================] - 1s 54ms/step - loss: 0.0

In [17]:
model.evaluate([textcuboid_test,text_test],y_test1)

69/69 [==============================] - 0s 4ms/step - loss: 0.1473 - accuracy: 0.9689


[0.14726504683494568, 0.9689356088638306]

In [22]:
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('./save model/best_model_2-Channel Tensor(ELMo).h5'.format(i), custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)

69/69 [==============================] - 0s 4ms/step - loss: 0.0771 - accuracy: 0.9785


[0.07713154703378677, 0.9785290360450745]

In [23]:
y_pred = model.predict([textcuboid_test,text_test])
y_pred_labels = np.argmax(y_pred, axis=1)
macro_f1 = f1_score(y_test, y_pred_labels, average='macro')
print("Macro_F1 스코어:", macro_f1)
weighted_f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("weighted_F1 스코어:", weighted_f1)

69/69 [==============================] - 0s 3ms/step
Macro_F1 스코어: 0.9451209948786785
weighted_F1 스코어: 0.9784417821519337
